In [14]:
class knn:
    dados = []
    labels = []
    classes = []
  
    def __init__(self, **args):

        self.k = 3
        
        if 'k' in args:
            self.k = args['k']

    def calculo_de(self, v1, v2):
        if(len(v1) != len(v2)):
            raise Exception('O Tamanho dos vetores devem ser iguais')
        soma = 0
        for i in range(len(v1)):
            soma += pow(float(v2[i]) - float(v1[i]), 2)
        return pow(soma, 0.5)
    
    def calcular_distancias(self, matriz, classes, vetor):    
        distancias = []
        for i in range(len(matriz)):
            distancias.append([self.calculo_de(matriz[i], vetor),classes[i]])
        return distancias

    def achar_classes(self, labels):
        classes = set()
        for label in labels:
            classes.add(label)
        return classes

    def predict(self, matriz):
        labels = []
        for linha in matriz:        
            distancias_linha = self.calcular_distancias(self.dados, self.labels , linha)            
            labels.append(self.contar_classes(distancias_linha))            
        return labels
    
    def montar_mapa_classes(self, kdistancias):
        mapa = {}
        for linha in kdistancias:
            if linha[1] in mapa:
                mapa[linha[1]] += 1
            else:
                mapa.update({linha[1]:1})   
        return mapa
    
    def fit(self,X, y):        
        self.dados = X
        self.labels = y       
        return self
    
    def score(self, X, y=None):
        predito = self.predict(X)
        acertos = 0
        for y_teste, y_predito in zip( y, predito):
            if y_teste == y_predito:
                acertos += 1
        return acertos / len(y)
        
    
    def get_params(self, deep = False):
        return {'k':self.k}
       
    
    def contar_classes(self, distancias):        
        distancias.sort()
        mapa = self.montar_mapa_classes(distancias[:self.k])           
        s = [(k, mapa[k]) for k in sorted(mapa, key = mapa.get, reverse=True)]        
        return s[0][0]

In [63]:
import csv
from random import seed
from random import randrange


dados=[]
target=[]

with open('iris.data', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
#     cabecalho = next(csv_reader)
    for line in csv_reader:
        if len(line) != 0:        
            dados.append(line[0:4])
            target.append(line[4])


 
def train_test_split(dataset, target, per_teste=0.60, semente = 42):
    seed(semente)      
    X_train = list(dataset)
    x_test = list()
    y_train = list(target)
    y_test = list()
    test_size = per_teste * len(dataset)   
    
    while len(x_test) < test_size:
        index = randrange(len(X_train))
        x_test .append(X_train.pop(index))
        y_test.append(y_train.pop(index))    
    
    return X_train, x_test, y_train, y_test


X_train, x_test, y_train, y_test = train_test_split(dados, target, per_teste = 0.4)



In [64]:
#  modelo = knn(k= 3, dados = X_train, labels = y_train)
modelo = knn(k= 3)
modelo.fit(X_train, y_train)

print(X_train)

[['5.1', '3.5', '1.4', '0.2'], ['4.7', '3.2', '1.3', '0.2'], ['4.6', '3.1', '1.5', '0.2'], ['5.0', '3.6', '1.4', '0.2'], ['5.4', '3.9', '1.7', '0.4'], ['4.8', '3.4', '1.6', '0.2'], ['4.8', '3.0', '1.4', '0.1'], ['4.3', '3.0', '1.1', '0.1'], ['5.7', '4.4', '1.5', '0.4'], ['5.4', '3.4', '1.7', '0.2'], ['5.1', '3.7', '1.5', '0.4'], ['4.6', '3.6', '1.0', '0.2'], ['4.8', '3.4', '1.9', '0.2'], ['5.4', '3.4', '1.5', '0.4'], ['5.2', '4.1', '1.5', '0.1'], ['5.5', '4.2', '1.4', '0.2'], ['4.9', '3.1', '1.5', '0.1'], ['5.0', '3.2', '1.2', '0.2'], ['5.5', '3.5', '1.3', '0.2'], ['4.4', '3.0', '1.3', '0.2'], ['5.0', '3.5', '1.3', '0.3'], ['4.5', '2.3', '1.3', '0.3'], ['4.4', '3.2', '1.3', '0.2'], ['5.1', '3.8', '1.9', '0.4'], ['4.8', '3.0', '1.4', '0.3'], ['4.6', '3.2', '1.4', '0.2'], ['5.3', '3.7', '1.5', '0.2'], ['5.0', '3.3', '1.4', '0.2'], ['6.4', '3.2', '4.5', '1.5'], ['6.9', '3.1', '4.9', '1.5'], ['5.5', '2.3', '4.0', '1.3'], ['5.7', '2.8', '4.5', '1.3'], ['6.3', '3.3', '4.7', '1.6'], ['4.9', '

In [66]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

predict = modelo.predict(x_test)

print(metrics.classification_report(y_test, predict))
print(metrics.confusion_matrix(y_test, predict))
print(modelo.score(x_test,y_test))

score = cross_val_score(modelo, dados, target, cv=10)
print(score.mean())


# metrics.

                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        22
Iris-versicolor       1.00      0.89      0.94        19
 Iris-virginica       0.90      1.00      0.95        19

      micro avg       0.97      0.97      0.97        60
      macro avg       0.97      0.96      0.96        60
   weighted avg       0.97      0.97      0.97        60

[[22  0  0]
 [ 0 17  2]
 [ 0  0 19]]
0.9666666666666667
0.9466666666666667
